In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder

In [2]:
df = pd.read_csv('../data/survey_data_before_pre_processing.csv')

### Remove Refused Persons

General rundown here is to calculate the amount of refusals per person as a percentage of total questions refused.

Find some cutoff and drop those people

### Convert all Refused to NaN

We will be imputing these, but need them to be nulls

### Ordinal Encode

There are a lot these.....

In [3]:
# Replacing nulls in these columns with 'none'
df['harm_plants_animals'].fillna('none', inplace=True)
df['reg_CO2_pollutant'].fillna('none', inplace=True)
df['reg_utilities'].fillna('none', inplace=True)


In [4]:
df[['generation', 'educ_category', 'income_category', 'harm_dev_countries',
              'harm_future_gen', 'harm_plants_animals', 'when_harm_US', 'reg_CO2_pollutant',
              'reg_utilities', 'fund_research', 'discuss_GW']].isnull().sum()

generation             0
educ_category          0
income_category        0
harm_dev_countries     0
harm_future_gen        0
harm_plants_animals    0
when_harm_US           0
reg_CO2_pollutant      0
reg_utilities          0
fund_research          0
discuss_GW             0
dtype: int64

In [5]:
# Ordinal encoding these features

cats = [['Greatest (Before 1928)','Silent (1928 - 1945)',
         'Baby Boomers (1946 - 1964)',
         'Generation X (1965 - 1980)',
         'Millennials (1981 - 1996)',
         'iGen\u200e/Gen Z (1997 - )'],
        ['Less than high school',
         'High school',
         'Some college',
         'Bachelor\'s degree or higher'],
        ['Less than $50,000',
         '$50,000 to $99,999',
         '$100,000 or more'],
        ['No',"Don't know", 'Yes', 'Refused', 'none'],
        ['No',"Don't know", 'Yes', 'Refused', 'none'],
        ['No',"Don't know", 'Yes', 'Refused', 'none'],
        ['Never', 'In the Future', 'Now', 'Refused'],
        ['Oppose', 'Support', 'none', 'Refused'],
        ['Oppose', 'Support', 'none', 'Refused'],
        ['Oppose', 'Support', 'none', 'Refused']
        ,['Never', 'At All', 'Refused']]

o_enc = OrdinalEncoder(categories=cats) 

o_enc.fit(df[['generation', 'educ_category', 'income_category', 'harm_dev_countries',
              'harm_future_gen', 'harm_plants_animals', 'when_harm_US', 'reg_CO2_pollutant',
              'reg_utilities', 'fund_research', 'discuss_GW']])

X_ord = o_enc.transform(df[['generation', 'educ_category', 'income_category', 'harm_dev_countries',
              'harm_future_gen', 'harm_plants_animals', 'when_harm_US', 'reg_CO2_pollutant',
              'reg_utilities', 'fund_research', 'discuss_GW']])

pd.DataFrame(X_ord)

,0,1,2,3,4,5,6,7,8,9,10
0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0
1,2.0,0.0,0.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0
2,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,3.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0
4,4.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
21859,2.0,2.0,0.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0
21860,2.0,0.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0
21861,4.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0
21862,3.0,2.0,0.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,0.0


In [6]:
df['generation_ord'] = [x[0] for x in X_ord]
df['educ_category_ord'] = [x[1] for x in X_ord]
df['income_category_ord'] = [x[2] for x in X_ord]
df.drop(['generation', 'educ_category', 'income_category'], axis=1, inplace=True)
df.head()

,wave,year,weight_wave,weight_aggregate,happening,cause_other_text,cause_recoded,sci_consensus,worry,harm_personally,...,house_ages13to17,house_ages18plus,house_type,house_own,age_category_35_54,age_category_55_plus,children,generation_ord,educ_category_ord,income_category_ord
0,2008 Nov,2008,0.54,0.294402,Yes,NaN,Caused mostly by human activities,Most scientists think global warming is happening,Somewhat worried,Only a little,...,0,3,One-family house detached from any other house,Owned by you or someone in your household,0,1,0,1.0,1.0,1.0
1,2008 Nov,2008,0.85,0.463411,Don't know,NaN,Caused mostly by human activities,Don't know enough to say,Not very worried,Only a little,...,0,2,Mobile home,Rented,1,0,0,2.0,0.0,0.0
2,2008 Nov,2008,0.49,0.267143,Don't know,NaN,Caused mostly by natural changes in the enviro...,There is a lot of disagreement among scientist...,Not at all worried,Not at all,...,0,2,One-family house detached from any other house,Owned by you or someone in your household,1,0,0,2.0,3.0,0.0
3,2008 Nov,2008,0.29,0.158105,Yes,NaN,Caused mostly by natural changes in the enviro...,Most scientists think global warming is happening,Somewhat worried,Only a little,...,0,2,One-family house detached from any other house,Owned by you or someone in your household,0,1,0,1.0,3.0,2.0
4,2008 Nov,2008,1.29,0.703294,Yes,NaN,Caused mostly by human activities,There is a lot of disagreement among scientist...,Somewhat worried,Don't know,...,0,2,One-family house detached from any other house,Owned by you or someone in your household,0,0,0,4.0,2.0,1.0


In [7]:
dogs = [
# year    
    [2008, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018],

# wave    
    ['2008 Nov', '2010 Jan', '2010 Jun', '2011 May', '2011 Nov', 
     '2012 Mar', '2012 Sep', '2013 Apr', '2013 Nov', '2014 Apr',
     '2014 Oct', '2015 Mar', '2015 Oct', '2016 Mar', '2016 Nov',
     '2017 May', '2017 Oct', '2018 Mar', '2018 Dec',]  
]

enc = OrdinalEncoder(categories=dogs)
enc.fit(df[['year', 'wave',]]) 
X_ord = enc.transform(df[['year', 'wave',]]) 

# harm_personally, harm_US, cause_recoded, worry

In [8]:
df['year_ord'] = [x[0] for x in X_ord]
df['wave_ord'] = [x[1] for x in X_ord]
df.drop(['year', 'wave'], axis=1, inplace=True)
df.head()

,weight_wave,weight_aggregate,happening,cause_other_text,cause_recoded,sci_consensus,worry,harm_personally,harm_US,harm_dev_countries,...,house_type,house_own,age_category_35_54,age_category_55_plus,children,generation_ord,educ_category_ord,income_category_ord,year_ord,wave_ord
0,0.54,0.294402,Yes,NaN,Caused mostly by human activities,Most scientists think global warming is happening,Somewhat worried,Only a little,A moderate amount,Yes,...,One-family house detached from any other house,Owned by you or someone in your household,0,1,0,1.0,1.0,1.0,0.0,0.0
1,0.85,0.463411,Don't know,NaN,Caused mostly by human activities,Don't know enough to say,Not very worried,Only a little,Refused,Yes,...,Mobile home,Rented,1,0,0,2.0,0.0,0.0,0.0,0.0
2,0.49,0.267143,Don't know,NaN,Caused mostly by natural changes in the enviro...,There is a lot of disagreement among scientist...,Not at all worried,Not at all,Not at all,No,...,One-family house detached from any other house,Owned by you or someone in your household,1,0,0,2.0,3.0,0.0,0.0,0.0
3,0.29,0.158105,Yes,NaN,Caused mostly by natural changes in the enviro...,Most scientists think global warming is happening,Somewhat worried,Only a little,Only a little,Yes,...,One-family house detached from any other house,Owned by you or someone in your household,0,1,0,1.0,3.0,2.0,0.0,0.0
4,1.29,0.703294,Yes,NaN,Caused mostly by human activities,There is a lot of disagreement among scientist...,Somewhat worried,Don't know,Don't know,Don't know,...,One-family house detached from any other house,Owned by you or someone in your household,0,0,0,4.0,2.0,1.0,0.0,0.0


### Train Test Split

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
y = df.happening
X = df[[cols for cols in df.columns if cols != "happening"]]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                   test_size = 0.20,
                                                   random_state = 21)

### Iteratively Impute Nulls

Now all of our data is ordinal encoded or dummied, iteratively impute the nulls

In [12]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [13]:
# identify numerical columns
cols = X_train.describe().columns

In [14]:
# iterative impute on numerical columns
pd.DataFrame(IterativeImputer(initial_strategy='mean',
                              add_indicator=True,
                              random_state = 21
                             ).fit_transform(X_train[cols]))\
            .head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0.8405,0.745574,1.0,49.0,5.0,3.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,3.0,1.0,1.0,6.0,12.0
1,0.9399,0.833747,1.0,70.0,5.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,6.0,12.0
2,1.4493,1.687926,0.0,31.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4.0,1.0,0.0,5.0,9.0
3,0.4640,0.659545,0.0,50.0,1.0,4.0,0.0,0.0,1.0,1.0,2.0,1.0,0.0,2.0,2.0,1.0,1.0,4.0,8.0
4,0.6804,0.635573,0.0,70.0,5.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,1.0,3.0,2.0,6.0,11.0
5,0.8700,0.474315,0.0,30.0,2.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0
6,2.1900,2.523183,1.0,23.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4.0,2.0,0.0,1.0,2.0
7,0.3025,0.341518,0.0,58.0,1.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,2.0,3.0,2.0,4.0,7.0
8,0.8352,0.755644,0.0,47.0,5.0,5.0,1.0,0.0,1.0,0.0,3.0,1.0,0.0,2.0,3.0,3.0,2.0,8.0,16.0
9,0.3647,0.386238,1.0,74.0,5.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,1.0,3.0,1.0,9.0,18.0


### Standard Scale

Will let us explore knn or clustering if wanted